## Audio classification in Julia
#### Dataset: Urban Sounds dataset 
#### Author - Yash Bhalgat
10 classes -

In [1]:
classes = Dict("air_conditioner" => 1, "car_horn" => 2, "children_playing" => 3, "dog_bark" => 4, "drilling" => 5,
                "engine_idling" => 6, "gun_shot" => 7, "jackhammer" => 8, "siren" => 9, "street_music" => 10)

Dict{String,Int64} with 10 entries:
  "gun_shot"         => 7
  "siren"            => 9
  "air_conditioner"  => 1
  "children_playing" => 3
  "dog_bark"         => 4
  "engine_idling"    => 6
  "car_horn"         => 2
  "street_music"     => 10
  "drilling"         => 5
  "jackhammer"       => 8

You may want to install some of these packages.

In [ ]:
using Images, DSP, PyPlot, MFCC, WAV, MP3

In [4]:
# For handling files
using Glob

Loading the audio filenames in the dataset.

In [7]:
mp3_files = []
for k in keys(classes)
    mp3_files = vcat(mp3_files, glob("*.mp3", string("UrbanSound/data/", k, "/")))
end

In [8]:
wav_files = []
for k in keys(classes)
    wav_files = vcat(wav_files, glob("*.wav", string("UrbanSound/data/", k, "/")))
end

## Feature extraction
`MFCC.jl` has a `feacalc(s; sr=sr)` function call which seemed to run VERY slowly on my laptop. And `MusicProcessing.jl` did not compile for this Julia version (there were other issues with MFCC and other versions).

I wrote the following method to calculate the Mel features, which works faster. There are some hacky parameters like the FRAME_LENGTH, FRAME_INTERVAL and nfilts, which can be adjusted if it throws any errors.



In [9]:
function get_features(s, sr)
    FRAME_LENGTH = 0.025
    FRAME_INTERVAL = 0.010
    frames = powspec(s, sr; wintime=FRAME_LENGTH, steptime=FRAME_INTERVAL);
    energies = log.(sum(frames', 2));
    fbanks = audspec(frames, sr; nfilts=40, fbtype=:mel)';
    fbanks = hcat(fbanks, energies);
    fbank_deltas = deltas(fbanks);
    fbank_deltadeltas = deltas(fbank_deltas);
    features = hcat(fbanks, fbank_deltas, fbank_deltadeltas);
#     imshow(features)
    return features
end

get_features (generic function with 1 method)

### Other alternative: Spectrogram
Or you can just take the spectrogram of the signal. But I think **better feature extraction leads to better learning!** So, use the above one.

For spectrogram-based feature extraction, do the following:
``` 
S = spectrogram(s, convert(Int, ceil(20e-3*sr)), convert(Int, ceil(10e-3*sr)); window=hanning)
features = log10.(power(S)+10e-6)
```

Here, again, the numbers `20e-3` and `10e-3` are hacky. In the second line, I've added `10e-6` because otherwise it leads to NaN error due to the `log` function.

## Actual stuff
Now, we start with the actual training!

Firstly, different functions needed for loading `.wav` and `.mp3` files. That explains the following 4-5 cells. 

Secondly, I've sampled first `65489` samples from each signal. If a signal is shorter than that, I've padded it to get it to that length. This leads to a fixed sized output from the `get_features` function defined above. That is then resized to `(128, 128)`. 

*Note: You actually don't even need to resize it, it will work anyway!* 

In [ ]:
audio = wavread(wav_files[1], subrange=65489)
s = audio[1][:,1]
sr = audio[2]
# S = spectrogram(s, convert(Int, ceil(20e-3*sr)),
#                         convert(Int, ceil(10e-3*sr)); window=hanning)
# features = log10.(power(S)+10e-6)

features = get_features(s, sr)
features = imresize(features, (128,128))
labels = [classes[split(wav_files[1], "/")[3]]]
sequences = [features]

I do this, because initial there was no resizing to the same size `(128, 128)`. Hence, all the features would be of different sizes because of different samplerates. But if we are resizing it, then you might as well allocate an arrays of zeros and go ahead from there. Nevermind for now. Let's process all the audio files.

You can see that there is lot of `try-catch` going on here. That's because I am not able to load some of the audio files using the Julia wrappers. If you do, it might improve the accuracy a little bit...

In [11]:
for file_name in wav_files[2:end]
    try
        try
            ### If signal is longer than 65489, cut it!
            audio = wavread(file_name, subrange=65489)
            s = audio[1][:,1]
            sr = audio[2]
        catch
            ### Padding with zeros if signal is shorter!
            audio = wavread(file_name)
            s = zeros(65489,)
            s[1:size(audio[1],1)] = audio[1][:,1]
            sr = audio[2]
        end
        
        ### If you want Spectrogram features, uncomment the following part 
        ### and comment the line get_features.
#         try
#             S = spectrogram(s, convert(Int, ceil(20e-3*sr)),
#                         convert(Int, ceil(10e-3*sr)); window=hanning)
#         catch
#             @show sr
#             continue
#         end
#         features = log10.(power(S)+10e-6)
        
        features = get_features(s, sr)
        
        features = imresize(features, (128,128))
        push!(labels, classes[split(file_name, "/")[3]])
        push!(sequences, features)
    catch
        @show file_name
    end
end

file_name = "UrbanSound/data/air_conditioner/195969.wav"
file_name = "UrbanSound/data/children_playing/193697.wav"
file_name = "UrbanSound/data/children_playing/193698.wav"
file_name = "UrbanSound/data/children_playing/193699.wav"
file_name = "UrbanSound/data/children_playing/204408.wav"
file_name = "UrbanSound/data/children_playing/36429.wav"
file_name = "UrbanSound/data/dog_bark/175915.wav"
file_name = "UrbanSound/data/dog_bark/175917.wav"
file_name = "UrbanSound/data/dog_bark/176783.wav"
file_name = "UrbanSound/data/dog_bark/180256.wav"
file_name = "UrbanSound/data/dog_bark/180257.wav"
file_name = "UrbanSound/data/engine_idling/176787.wav"
file_name = "UrbanSound/data/drilling/161129.wav"
file_name = "UrbanSound/data/drilling/180029.wav"
sr = 22050.0f0
file_name = "UrbanSound/data/jackhammer/88466.wav"


In [12]:
size(sequences)

(909,)

In [13]:
first_size = [size(seq,1) for seq in sequences];
sec_size = [size(seq,2) for seq in sequences];

Again, did this because initially there was no resizing to a constant size. Hence, wanted to see on an average what the size turns out to be.

In [14]:
@show mean(first_size)
@show mean(sec_size)

mean(first_size) = 128.0
mean(sec_size) = 128.0


128.0

In [ ]:
for file_name in mp3_files
    try
        try
            audio = load(file_name)
            s = convert(Array{Float64,1}, audio.data[:,1][1:65489])
            sr = audio.samplerate
        catch
            audio = load(file_name)
            s = zeros(65489,)
            s[1:size(audio.data,1)] = convert(Array{Float64,1}, audio.data[:,1])
            sr = audio.samplerate
        end
        
#         try
#             S = spectrogram(s, convert(Int, ceil(20e-3*sr)),
#                         convert(Int, ceil(10e-3*sr)); window=hanning)
#         catch
#             @show sr
#             continue
#         end
#         features = log10.(power(S)+10e-6)
        
        features = get_features(s, sr)
        
        features = imresize(features, (128,128))
        push!(labels, classes[split(file_name, "/")[3]])
        push!(sequences, features)
    catch
        @show file_name
    end
end

In [16]:
first_size = [size(seq,1) for seq in sequences];
sec_size = [size(seq,2) for seq in sequences];

In [17]:
@show mean(first_size)
@show mean(sec_size)

mean(first_size) = 128.0
mean(sec_size) = 128.0


128.0

### Data Normalization
This is very important. Different filter banks have different **unnormalized** coefficients! So, if you don't normalize, there are some features in the order of `10e-2` and some in the order of `10e+2`.

The following function I wrote will do it for you.

*Note: Btw, there is ZCA-whitening you can do too, if you would like. But this one works almost the same.*

In [18]:
function normalize_features(features)
    features_normalized = zeros(size(features))
    for r in 1:size(features,1)
        for c in 1:size(features,2)
            avg = mean(features[r,c,:])
            stddev = std(features[r,c,:])
            features_normalized[r,c,:] = (features[r,c,:].-avg)/stddev
        end
    end
    return features_normalized
end

normalize_features (generic function with 1 method)

In the next cell, I am converting the array of features into a 3D-matrix.

In [19]:
seq_array = reshape(sequences[1], 128, 128, 1)
for j in 2:size(sequences,1)
    seq_array = cat(3, seq_array, reshape(sequences[j], 128, 128, 1))
end

In [20]:
size(seq_array)

(128, 128, 1047)

Remember I mentioned `feacalc` works very slow? You can uncomment and run the next cell. There is not information in these features as shown in the image generated, hence I used my custom-built function.

In [21]:
# mel_features = feacalc(s; sr=sr);
# imshow(mel_features[1])

Normalize the data.

In [22]:
seq_normalized = normalize_features(seq_array);

In [23]:
seq_normalized = reshape(seq_normalized, (128, 128, 1, 1047))
size(seq_normalized)

(128, 128, 1, 1047)

In [24]:
using Flux
using Flux: onehotbatch, crossentropy, throttle
using Base.Iterators: repeated, partition

### Train-validation/test split
Since there is no default train-test set, I randomly split the data a set of 1000 train samples and remaining validation/test samples.

Line no. `10` splits the train data into batches of size 10.

Don't forget to transfer all the data to the gpu!

In [25]:
idx = shuffle(1:size(seq_normalized,4))
shuffled_data = seq_normalized[:,:,:,idx]
shuffled_labels = labels[idx]
trainX = shuffled_data[:,:,:,1:1000]
trainY = shuffled_labels[1:1000]
testX = shuffled_data[:,:,:,1001:end] |> gpu
testY = shuffled_labels[1001:end]
one_hot_train = onehotbatch(trainY, 1:10)
one_hot_test = onehotbatch(testY, 1:10) |> gpu
train_data = [(trainX[:,:,:,i], one_hot_train[:,i])
         for i in partition(1:size(trainX,4), 10)]
train_data = gpu.(train_data);

The next cell shows the convolutional network I used. Feel free to modify it. After a lot of tuning, for me, this worked the best.

Test accuracy on 47 samples (too less) ~ 41%.

In [40]:
m = Chain(
  Conv((2,2), 1=>16, relu),
  x -> maxpool(x, (2,2)),
  Conv((4,4), 16=>8, relu),
  x -> maxpool(x, (2,2)),
  Conv((8,8), 8=>8, relu),
  x -> maxpool(x, (2,2)),
  x -> reshape(x, :, size(x, 4)),
  Dense(968, 10), softmax) |> gpu

Chain(Conv((2, 2), 1=>16, NNlib.relu), #35, Conv((4, 4), 16=>8, NNlib.relu), #36, Conv((8, 8), 8=>8, NNlib.relu), #37, #38, Dense(968, 10), NNlib.softmax)

In [41]:
loss(x, y) = crossentropy(m(x), y)

loss (generic function with 1 method)

The next cell - because Julia 0.6 Flux does not have the `onecold` function (Julia 1.0 version has it). Just wrote one for using here.

In [42]:
function one_cold(preds)
    return [indmax(preds[:,i]) for i in 1:size(preds,2)]
end

one_cold (generic function with 1 method)

In [43]:
accuracy(x, y) = mean(one_cold(m(x)) .== one_cold(y))

accuracy (generic function with 1 method)

In [44]:
evalcb = throttle(() -> @show(accuracy(testX, one_hot_test)), 10)
opt = ADAM(params(m))

(::#93) (generic function with 1 method)

In [46]:
using Flux: @epochs
@epochs 15 Flux.train!(loss, train_data, opt, cb = evalcb)

INFO: Epoch 1


accuracy(testX, one_hot_test) = 0.40425531914893614
accuracy(testX, one_hot_test) = 0.3829787234042553
accuracy(testX, one_hot_test) = 0.40425531914893614
accuracy(testX, one_hot_test) = 0.3829787234042553
accuracy(testX, one_hot_test) = 0.40425531914893614


INFO: Epoch 2


accuracy(testX, one_hot_test) = 0.3617021276595745
accuracy(testX, one_hot_test) = 0.40425531914893614
accuracy(testX, one_hot_test) = 0.3617021276595745


INFO: Epoch 3


...

accuracy(testX, one_hot_test) = 0.2978723404255319
accuracy(testX, one_hot_test) = 0.3404255319148936
accuracy(testX, one_hot_test) = 0.3191489361702128
accuracy(testX, one_hot_test) = 0.3617021276595745


INFO: Epoch 14


accuracy(testX, one_hot_test) = 0.2765957446808511
accuracy(testX, one_hot_test) = 0.3191489361702128
accuracy(testX, one_hot_test) = 0.2978723404255319
accuracy(testX, one_hot_test) = 0.40425531914893614


INFO: Epoch 15


accuracy(testX, one_hot_test) = 0.3191489361702128
accuracy(testX, one_hot_test) = 0.2978723404255319
accuracy(testX, one_hot_test) = 0.2765957446808511
accuracy(testX, one_hot_test) = 0.3617021276595745


In [ ]:
accuracy(testX, one_hot_test)

And we are done! :)